In [2]:
from transformers import BertTokenizer,BertModel
import torch.nn as nn
import torch
from transformers import logging
logging.set_verbosity_error()

class MiniRBTClassifier(nn.Module):
    def __init__(self, model_name, num_labels=3):
        super(MiniRBTClassifier, self).__init__()
        self.encoder = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)

tokenizer = BertTokenizer.from_pretrained("hfl/minirbt-h256")
model = MiniRBTClassifier(model_name="hfl/minirbt-h256")  # 重新实例化模型

# 加载模型权重
model.load_state_dict(torch.load('minirbt_sentiment_model.pth',map_location='cpu',weights_only=False))
model.eval()  # 设置为评估模式


MiniRBTClassifier(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elemen

In [3]:
def predict_sentiment(text, model, tokenizer, max_length=512):
    encoding = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # 进行推理
    with torch.no_grad():  # 关闭梯度计算以节省内存
        output = model(input_ids=input_ids, attention_mask=attention_mask)

    # 获取模型的预测结果
    predicted_class = torch.argmax(output, dim=1).item()
    return predicted_class


In [15]:
# 输入一段文本
text = "谁送我的1.6米的大棕熊啊……赶紧认领下[衰][衰][衰][衰][衰][衰][衰][衰][衰][衰][衰][衰][衰]"

# 获取情感分类的预测结果
predicted_class = predict_sentiment(text, model, tokenizer)

# 标签映射
id2label = {0: '积极', 1: '中性', 2: '消极'}

# 输出预测结果
print(f"预测的情感标签是: {id2label[predicted_class]}")


预测的情感标签是: 消极
